# Usage of EC for the Traveling Salesman Problem
## CI2025_Lab2
**K/BIDI Thomas**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm

## Problems

We are trying to solve the Traveling Salesman Problem (and some variants of it) using **Evolutionnary Computation algorithms**.  
To achieve either an optimal or "good enough" solution for each problems we're gonna explore an implementation of the EC with the following definitions and properties : 
(We're using the terms of the classic TSP, which consists of finding a minimal-cost hamiltonian cycle in an undirected complete graph, where each edge's weight is the distance between the two cities it connects. For the R1 and R2 variants, the ideas are the same, except the concept of distance cannot be applied to them [matrix are not always symmetric, triangular inequality does not necessary hold].)

- **Fitness** : The fitness represent the sum of distances between the cities of the solution (which includes the distance between the final and first cities)
- **Individual** : An individual is a sequence of cities describing the hamiltonian cycle, it holds this properties $$|i| = |V|$$ with  $i$ an individual, and $V$ the set of vertices of the graph. So in our case, a gene is a city of the cycle.
- **Population** : The population is a set of individuals.

In [ ]:
# Here was my first implementation of the fitness
def fitness(solution, problem):
    result = problem[solution[-1], solution[0]]
    for i in range(len(solution) - 1):
        result += problem[solution[i], solution[i+1]]
    return result

def population_fitness(population, problem):
    # Return the fitness for every individuals in the population
    pop_fitness = [fitness(i, problem) for i in population]
    return pop_fitness

We're going to implement various functions, the idea is to explore the fitness space and find a pretty good, if not *optimal*, solution.  

Firstly, let's define the main function we're going to use, this function will create the initial population and realize the different steps of EC :
- Initialize the population randomly
- Realize the selection of the parents
- Realize the crossovers and mutations
- Select the offspring 
- Keep track of the fitness over each generations and plot a graph of it at the end
- Keep track of the overall best solutions and fitness and return them

In [ ]:
# Function used for early stopping, here we will stop when the generation best fitness do not move during (10% * N_GENS) consecutives generations.
def on_plateau_for_K(fitness_list, K):
    if (len(fitness_list) < K):
        return False
    last_K = fitness_list[-K:]
    return np.all(last_K == last_K[0])

In [ ]:
def generation(problem, N_GENS, POP_SIZE, parent_selection, survivor_selection, offsprings_generation, mutation, mu_coeff, l_coeff, problem_name=""):
    """
    problem : the matrix of distances between the cities
    N_GENS : the number of generations on which the evolutionary process will be applied
    POP_SIZE : the number of individuals in the original population
    parent_selection : function used to select the parent individuals
    offsprings_generation : function performing recombination and mutation on the parent population to generate the offspring
    survivor_selection : function used to select the new population from the offspring
    mutation : function applied during offspring generation
    mu_coeff : gives mu (the portion of the population selected as parents) when multiplied by POP_SIZE
    l_coeff : gives lambda (the offspring and new population size) when multiplied by POP_SIZE
    problem_name : name of the problem to display on the generated graph
    """
    # Random population generation
    population = np.array([np.random.permutation(np.arange(problem.shape[0])) for _ in range(POP_SIZE)])
    
    mu = int(POP_SIZE * mu_coeff)
    l = int(POP_SIZE * l_coeff)

    # Keep track of the best individual in all generations
    best_individual = population[0]
    best_fitness = fitness(best_individual, problem)
    best_fitness_generation = -1 # Generation in which the best_fitness appeared

    # Keep tracks of all the generations best fitness to plot the graph of its evolution
    fitness_list = []
    plateau_size = int(N_GENS * 0.1) # 10%
    num_gens = N_GENS # Value to keep plot only on the actual number of generations made

    # Use of TQDM to show a progress bar
    for gen in tqdm(range(N_GENS), desc="Evolution Computation", unit="gen"):
        # Usage of the given function to realize the different steps of the EC
        parents = parent_selection(population, mu, l, problem)
        offsprings = offsprings_generation(parents, l, problem, mutation)
        survivors = survivor_selection(parents, offsprings, mu, l)
        population = survivors

        generation_best_fitness = 1e30 # Suppose to be more that the worst possible fitness
        
        fitness_by_individual = population_fitness(population, problem)
        sort_indices = np.argsort(fitness_by_individual)
        generation_best_fitness = fitness_by_individual[sort_indices[0]]
        if (generation_best_fitness < best_fitness):
            best_individual = population[sort_indices[0]]
            best_fitness = generation_best_fitness
            best_fitness_generation = gen + 1
        
        fitness_list.append(generation_best_fitness)
        if (on_plateau_for_K(fitness_list, plateau_size)):
            print(f"Early stopping due to best generation fitness not moving for {plateau_size} generations")
            num_gens = gen + 1
            break

    print(f"The best fitness is {best_fitness} and it was found in generation : {best_fitness_generation}")
    # We plot the graph of the fitness over the generations
    plt.plot(range(num_gens), fitness_list)
    # We get the first value closer to the best fitness and add a point here
    plt.scatter(best_fitness_generation, best_fitness, color="red", label="Best fitness")
    plt.xlabel("Generations")
    plt.ylabel("Fitness")
    plt.title(f"Evolution of fitness over generations ({problem_name})")
    plt.legend()
    plt.show()

    return best_individual, best_fitness

### Parent selection

For the parent selection, multiple options exists and we are going to implement 2 of them :
- **Uniform selection** : we randomly pick one the parents from the population
- **Best selection** : we sort the fitness values of all the individuals in the population and we select the $\mu$ first as the parents

In [ ]:
def uniform_selection(population, mu, l, problem):
    # We select mu individuals in the population randomly
    selection = np.random.choice(len(population), size=mu, replace=False)
    parents = np.array([population[i] for i in selection], dtype=int)
    return parents

In [ ]:
def best_selection(population, mu, l, problem):
    # We select the mu best individuals in the population
    fitness_by_individual = population_fitness(population, problem)
    sort_indices = np.argsort(fitness_by_individual)
    parents = np.array([population[sort_indices[i]] for i in range(mu)])
    return parents

### Crossover

For our problem we need a crossover method which can preserve the validity of the individuals, this condition make the n-point crossover a bad option, since it can generate individuals which are not an hamiltonian cycle.  
To answer this problem we're going to use the **Order Crossover (OX)**, the idea behind this crossover is that we select a subsequence of the first parent and we copy it at the same exact position inside the genotype of the offspring, then we complete the remaining empty genes with those of the second parent which are not already in the offspring's genotype.

In [ ]:
def order_crossover(parent1, parent2):
    # Choice of the start and end of the substring inside parent1
    start, end = np.sort(np.random.choice(len(parent1), size=2, replace=False))
    # Creation of the child
    child = [-1 for _ in range(len(parent1))]
    child[start:end] = parent1[start:end] # Copy of the subsequence in it

    # We create a mask and remove the already existing genes of the offsprings from the parent2
    mask = ~np.isin(parent2, child)
    parent2_filtered = parent2[mask]
    starting_index = 0 if start > 0 else end

    # We copy the remaining genes in the available place of the offspring's genotype
    for i in parent2_filtered:
        child[starting_index] = i
        starting_index += 1
        if (starting_index >= start and starting_index < end):
            starting_index = end

    return child

### Mutations

Every once in a while it is interesting for an offspring to mutate in order to favor the exploration of the fitness space. We are going to implement various simple mutation functions, which are the following ones : 
- **Swap mutation** : We randomly select 2 genes and we swap their positions.
- **Scramble mutation** : We randomly select 2 sequences and we swap their positions.
- **Insert mutation** : We randomly select a gene and move it.
- **Inversion mutation** : We randomly select a sequence in the individual and inverse it.
- **Random mutation** : We pick uniformely one of the previous mutation and apply it to the individual.

In [ ]:
# Swap mutation
def swap_mutation(individual):
    # Swap 2 elements in the individual
    gene1, gene2 = np.sort(np.random.choice(len(individual), size=2, replace=False))
    
    temp = individual[gene1]
    individual[gene1] = individual[gene2]
    individual[gene2] = temp

    return individual

In [ ]:
# Scramble mutation
def scramble_mutation(individual):
    # Swap 2 sequences in the individual
    start1, end1, start2, end2 = np.sort(np.random.choice(len(individual), size=4, replace=False))
    
    # Retrieve the sequences
    segment1 = individual[start1:end1]
    segment2 = individual[start2:end2]

    new_individual = np.concatenate([
        individual[:start1],
        segment2,
        individual[end1:start2],
        segment1,
        individual[end2:]
    ]).astype(int)

    return new_individual

In [ ]:
# Insert mutation
def insert_mutation(individual):
    # Randomly select the position of the gene to move
    gene_position, new_position = np.random.choice(len(individual), size=2, replace=False)
    
    # We delete and re-instert the gene
    gene = individual[gene_position]
    new_individual = np.delete(individual, gene_position)
    new_individual = np.insert(new_individual, new_position, gene)

    return new_individual

In [ ]:
# Inversion mutation
def inversion_mutation(individual):
    start, end = np.sort(np.random.choice(len(individual), size=2, replace=False))
    
    # We retrieve and inverse the segment
    segment = individual[start:end]
    inversed_segment = segment[::-1]

    # We concatenate the untouched parts with the reversed segments
    new_individual = np.concatenate([
        individual[:start],
        inversed_segment,
        individual[end:]
    ]).astype(int)

    return new_individual

In [ ]:
# Uniform selection in a set of mutations (swap, scramble, insert, inversion)
def random_mutation(individual):
    mutations = [swap_mutation, scramble_mutation, insert_mutation, inversion_mutation]
    choice = np.random.choice(len(mutations))
    return mutations[choice](individual)

### Offsprings generation

We need a function to generate the offsprings from the parents. We are going to select **uniformely** 2 parents in the given array and use the order crossover to produce an offspring. A *mutation* could appear with a probability of 40%.

In [ ]:
def offsprings_generation(parents, l, problem, mutation):
    # Generate lambda (l) offsprings from the parents population
    offsprings = []

    while len(offsprings) < l:
        # Parents are already the best of the population, we uniformely pick inside them
        p1, p2 = uniform_selection(parents, 2, -1, problem)
        offspring = order_crossover(p1, p2)
        if (np.random.rand() < 0.4): # Probability of 40% of a mutation
            offspring = mutation(offspring)
        offsprings.append(offspring)

    return np.array(offsprings)

### Survivor selection

We need to decide of a policy for the selection of the survivors which are going to replace the population for the next generation, here we are going to use this one :
- **Generational** : We replace all the population by the offsprings

In [ ]:
def generational_selection(parents, offsprings, mu, l):
    return offsprings # Only the offsprings are selected

### Problems list

All the problems we need to solve are in the lab2 subfolder, we're going to load them in a list. This will let us run all algorithms in a row and observe the results afterwards.

In [ ]:
problems = os.listdir("lab2/")
problems_info = {}

### Upgrade

I noticed that this implementation was a bit slow due to the computation of all the fitness values, we are going to use the Numpy's vectorization capabilities, which gives us access to the underlying C execution frame, to accelerate the **fitness** and **population_fitness** computation.

In [ ]:
# Vectorized version of the fitness and population_fitness
def fitness(solution, problem):
    return np.sum(problem[solution, np.roll(solution, -1)])

def population_fitness(population, problem):
    return np.sum(problem[population, np.roll(population, -1, axis=1)], axis=1)

### Final Test

We are now going to develop a small loop exploiting the list of problems and the differents function we developped earlier to search for the solutions of each problems.

Tweaking the parameters helps giving the best results, for this test I am going to set **N_GENS** at **5000** and the **POP_SIZE** at **500**.

In [ ]:
problems_info = {}

for problem_name in problems:
    problem = np.load(f"lab2/{problem_name}")
    best_solution, best_fitness = generation(problem, 5000, 500, best_selection, generational_selection, offsprings_generation, random_mutation, 0.05, 1, problem_name)
    problems_info[problem_name] = (best_solution, best_fitness)